In [ ]:
from src.utils import *

In [ ]:
# import wav data to keras dataset
import numpy as np
from keras.utils import audio_dataset_from_directory

# Load audio dataset from directory
train_dataset, validation_dataset = audio_dataset_from_directory(
    directory='data/path/train',
    validation_split=0.2,
    subset="both",
    labels="inferred",
    label_mode="categorical",
    seed=1337,
    batch_size=64,
    output_sequence_length=16000, 
)


In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
from tensorflow.keras.optimizers import Adam
from transformers import AutoFeatureExtractor, TFWav2Vec2ForSequenceClassification

# Load pre-trained Wav2Vec2 model and processor using AutoFeatureExtractor
model_name = "facebook/wav2vec2-base-960h"
tokenizer = AutoFeatureExtractor.from_pretrained(model_name)

# Load pre-trained Wav2Vec2 model
model = TFWav2Vec2ForSequenceClassification.from_pretrained(model_name)

In [ ]:
import tensorflow as tf
from transformers import AutoFeatureExtractor
from tqdm.notebook import tqdm

# Load Wav2Vec tokenizer
model_name = "facebook/wav2vec2-base-960h"
tokenizer = AutoFeatureExtractor.from_pretrained(model_name)
print("test")

# Define a function to tokenize audio data
def tokenize_dataset(dataset):
    tokenized_batches = []
    for batch in tqdm(dataset, total=len(dataset)):
        
        audio_batch, labels = batch
        # print(audio_batch)
        inputs = tokenizer(audio_batch, return_tensors="tf", padding=True, verbose=False, sampling_rate=16000)
        tokenized_batches.append((inputs.input_values, labels))
    return tokenized_batches

# Tokenize the train dataset
tokenized_train_dataset = tokenize_dataset(train_dataset)

# Tokenize the validation dataset
tokenized_validation_dataset = tokenize_dataset(validation_dataset)


import tensorflow as tf

# Convert tokenized batches into a generator function
def batch_generator(tokenized_batches):
    for input_values, labels in tokenized_batches:
        yield input_values, labels

# Create Keras batch datasets for train and validation datasets
keras_train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: batch_generator(tokenized_train_dataset),
    output_signature=(
        tf.TensorSpec(shape=(1, 32, 16000, 1), dtype=tf.float32),  # Input values
        tf.TensorSpec(shape=(32, 30), dtype=tf.int32)           # Labels
    )
)

keras_validation_dataset = tf.data.Dataset.from_generator(
    generator=lambda: batch_generator(tokenized_validation_dataset),
    output_signature=(
        tf.TensorSpec(shape=(1, 32, 16000, 1), dtype=tf.float32),  # Input values
        tf.TensorSpec(shape=(32, 30), dtype=tf.int32)           # Labels
    )
)



In [ ]:
X_train = tf.concat([i[0][0] for i in tokenized_train_dataset], axis=0)
X_val = tf.concat([i[0][0] for i in tokenized_validation_dataset], axis=0)

In [ ]:
y_train = tf.concat([i[1] for i in tokenized_train_dataset], axis=0)
y_val = tf.concat([i[1] for i in tokenized_validation_dataset], axis=0)

In [ ]:
X_train = X_train.numpy()
X_val = X_val.numpy()

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1])
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1])

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Dropout, Dense
from tensorflow.keras.layers import Bidirectional, LSTM, TimeDistributed


def get_SR_Model(num_classes: int):
    X_input = Input(shape=(16000, 1))
    X = Conv1D(filters=256,kernel_size=15,strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = Conv1D(filters=512,kernel_size=15,strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = LSTM(units=512, return_sequences=True)(X)
    X = LSTM(units=512, return_sequences=False)(X)
    X = Dense(num_classes, activation='softmax')(X)
    return Model(inputs=[X_input], outputs=[X])

model = get_SR_Model(2)



In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from transformers import TFWav2Vec2Model
from tensorflow.keras import Model, layers


class TFWav2Vec2Layer(layers.Layer):
    def __init__(self, **kwargs):
        super(TFWav2Vec2Layer, self).__init__(**kwargs)
        self.wav2vec_model = TFWav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        self.wav2vec_model.trainable = False

    def call(self, inputs, **kwargs):
        return self.wav2vec_model(inputs)["last_hidden_state"]


def get_wav2vec_classifier(num_classes: int):
    # Define input layer for tokenized input
    tokenized_input = layers.Input(shape=(None,), dtype=tf.int32)

    # Pass input through custom Wav2Vec2 layer
    hidden_states = TFWav2Vec2Layer()(tokenized_input)

    # Add classification layers
    x = layers.Dense(512, activation='relu')(hidden_states[:, 0, :])  # Take the first token's representation
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    # Define model
    model = Model(inputs=tokenized_input, outputs=x)

    return model


# Example usage:
model = get_wav2vec_classifier(num_classes=2)


In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                loss=losses.CategoricalCrossentropy(),
                metrics=[metrics.CategoricalAccuracy()])

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=64)